Functions to read devset and evalset and labels conversion

In [13]:
import csv
def read_devset():
    X = []
    y = []
    with open("datasets/development.csv", encoding="utf8") as dev_set:
        reader = csv.reader(dev_set)
        next(reader)
        for row in reader:
            X.append(row[0])
            y.append(row[1])
    
    return X, labels_to_ints(y)

def read_evalset():
    X = []
    with open("datasets/evaluation.csv", encoding="utf8") as eval_set:
        reader = csv.reader(eval_set)
        next(reader)
        for row in reader:
            X.append(row[0])
    
    return X
def labels_to_ints(labels):
    return list(map(lambda x : 1 if x == "pos" else 0, labels))

def ints_to_labels(labels):
    return list(map(lambda x : "pos" if x == 1 else "neg", labels))



In [14]:
X, y = read_devset()
X_ev = read_evalset()
len(X) + len(X_ev)

41077

In [25]:
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
import string
import re

def preprocess_word(text):
    text = text.lower()
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'url', text) # remove URLs
    text = re.sub(r"(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)", "email", text)
    text = re.sub('@[^\s]+', 'at_user', text) # remove usernames
    text = re.sub(r'#([^\s]+)', r'\1', text) # remove the # in #hashtag
    text = re.sub("'", "' ", text)
    return text

class LemmaTokenizer(object):
    def __init__(self):
        self.lemmatizer = WordNetLemmatizer()
    
    def __call__(self, text):
        lemmas = []
        regex = re.compile("[0-9]+")
        
        text = preprocess_word(text)
        for t in word_tokenize(text):
            t = t.strip()
            lemma = self.lemmatizer.lemmatize(t)
            if lemma not in string.punctuation and 3 < len(lemma) < 16 and not regex.match(lemma) and "@" not in lemma and "#" not in lemma:
                lemmas.append(lemma)
        return lemmas
    

In [26]:
from nltk.corpus import stopwords as sw
from sklearn.feature_extraction.text import TfidfVectorizer

stopwords = sw.words('italian') + ['quantum', "url", "at_user", "email"]
lemmaTokenizer = LemmaTokenizer()
# Trasforma le liste di token in numeri che vengono inseriti in una matrice di numpy float in modo da poterli analizzare
vectorizer = TfidfVectorizer(input="content", tokenizer=lemmaTokenizer, stop_words=stopwords)
tfidf_X = vectorizer.fit_transform(X)
tfidf_X


<28754x58892 sparse matrix of type '<class 'numpy.float64'>'
	with 1538645 stored elements in Compressed Sparse Row format>

PCA Process


In [9]:
import numpy as np
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=3000, random_state=42)
svd_X = svd.fit_transform(tfidf_X)
print(f"Total variance explained: {np.sum(svd.explained_variance_ratio_):.2f}")
np.shape(svd_X)

Total variance explained: 0.69


(28754, 3000)

In [31]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
X_train, X_test, y_train, y_test = train_test_split(tfidf_X, y)
clf = RandomForestClassifier(n_estimators=100, max_features="sqrt", criterion="gini")
clf.fit(X_train, y_train)
f1_score(y_test, clf.predict(X_test))

0.9371260421775379